In [6]:
# Importing modules
import pandas as pd
import os

# LTC_EVENTS = '../data/ltc_events_all_patients_07102021.tsv'
LTC_EVENTS = '../data/ltc_events_all_patients_ukbb45840.tsv'

events = pd.read_csv(LTC_EVENTS, sep = '\t')

# Print head
events.head()

,patient_id,event_date,read_2,LTC,LTC_abbrev,sex,YOB,age_at_event,count_per_LTC,count_distinct_LTC
0,1000014,09/06/2004,N20..,Polymyalgia Rheumatica,PMR,F,1946,58,1,1
1,1000014,12/07/2011,F451.,Glaucoma,glaucoma,F,1946,65,1,2
2,1000059,27/10/1995,N05z5,Osteoarthritis (excl spine),OA,M,1942,53,1,1
3,1000059,01/09/1998,N05..,Osteoarthritis (excl spine),OA,M,1942,56,2,1
4,1000059,24/05/1999,N05z5,Osteoarthritis (excl spine),OA,M,1942,57,3,1


In [ ]:
## collect the set of all LTC values for each patient. Each set forms a document

## dict
ltcs_per_patient = dict()

all_patients = events['patient_id'].unique()

print("{n} patients processed.".format(n=len(all_patients)))

list_of_LTC_lists = []

for patient in all_patients:
    ltc_list = events[events['patient_id'] == patient]['LTC'].unique().tolist()
    ltcs_per_patient[patient] = ltc_list
    list_of_LTC_lists.append(ltc_list)
#     print("patient {p}: \n{l}".format(p=patient, l = ltc_list))

print('complete list of lists:\n {l}'.format(l=list_of_LTC_lists))

In [8]:
import gensim.corpora as corpora
import gensim.models as models

id2word = corpora.Dictionary(list_of_LTC_lists)
# print(dictionary)

# Create Corpus
texts = list_of_LTC_lists

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View  -- limit 
print(corpus[:1][0][:30])


[(0, 1), (1, 1)]


In [ ]:
from pprint import pprint

# number of topics
num_topics = 10

# Build LDA model
lda_model = models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [10]:
import pickle 
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

# # Visualize the topics
pyLDAvis.enable_notebook()

gensimvis.prepare(lda_model, corpus, id2word)



# LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_'+str(num_topics))

# # # this is a bit time consuming - make the if statement True
# # # if you want to execute visualization prep yourself
# if 1 == 1:
#     LDAvis_prepared = pyLDAvis.prepare(lda_model, corpus, id2word)
#     with open(LDAvis_data_filepath, 'wb') as f:
#         pickle.dump(LDAvis_prepared, f)


c:\Users\Acer\OneDrive - Newcastle University\Work\CSC8639 - Thesis\MLTC-clustering-main\mltc\lib\site-packages\past\builtins\misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
c:\Users\Acer\OneDrive - Newcastle University\Work\CSC8639 - Thesis\MLTC-clustering-main\mltc\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9      0.133806 -0.018520       1        1  12.794749
0     -0.124757  0.084024       2        1  12.657961
1     -0.029318 -0.029910       3        1  11.733654
7     -0.016028 -0.036535       4        1   9.971266
4      0.055305  0.047581       5        1   9.679738
8     -0.044474  0.029548       6        1   8.897899
3     -0.066475 -0.073959       7        1   8.825909
6      0.014298  0.043288       8        1   8.798888
5     -0.029820 -0.069469       9        1   8.412648
2      0.107463  0.023951      10        1   8.227290, topic_info=                                                 Term          Freq  \
14                      Allergic and chronic rhinitis  36945.000000   
52                                  Chronic sinusitis  39547.000000   
3        Dermatitis (atopc/contact/other/unspecified)  55747.000000   
6   Primary Malignancy_Other Skin and subcutaneous...  12160.000000   
69                                 Anxiety and phobia  33291.000000   
..                                                ...           ...   
10                                        Spondylosis   1210.565817   
66  Post-traumatic stress and stress-related disor...   1256.433137   
28                                       Constipation   1004.158491   
26                                    Thyroid disease    974.896174   
38                           Gastritis and duodenitis    885.907396   

           Total Category  logprob  loglift  
14  36945.000000  Default  30.0000  30.0000  
52  39547.000000  Default  29.0000  29.0000  
3   55747.000000  Default  28.0000  28.0000  
6   12160.000000  Default  27.0000  27.0000  
69  33291.000000  Default  26.0000  26.0000  
..           ...      ...      ...      ...  
10  17189.068356  Topic10  -4.4373  -0.1555  
66  19935.200061  Topic10  -4.4001  -0.2665  
28  17657.989452  Topic10  -4.6243  -0.3693  
26  20763.310850  Topic10  -4.6538  -0.5609  
38  18500.239288  Topic10  -4.7496  -0.5412  

[535 rows x 6 columns], token_table=      Topic      Freq                             Term
term                                                  
198       2  0.140449  ADHD and hyperkinetic disorders
198       3  0.038304  ADHD and hyperkinetic disorders
198       5  0.459650  ADHD and hyperkinetic disorders
198       6  0.063840  ADHD and hyperkinetic disorders
198       7  0.268129  ADHD and hyperkinetic disorders
...     ...       ...                              ...
139       6  0.030006                         Vitiligo
139       7  0.238897                         Vitiligo
139       8  0.159265                         Vitiligo
139       9  0.159265                         Vitiligo
139      10  0.066937                         Vitiligo

[1890 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 1, 2, 8, 5, 9, 4, 7, 6, 3])